# Intro
Supervised Learning
- Regression
- Classification

Date Range: 

Features: 
- Prediction: Date, Vacination rate, Infection number --> ICU Utilization predict in %
- Classification :  Date, Vacination number (age group), Infection number (age Group) --> COVID Risk (low, high)

# Prerequisite

In [ ]:
import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models

# Loading DataSet from Github

In [ ]:
# URL should be https://raw.githubusercontent.com/username/repository/branch/filename.csv so no download needed

# Cases, Age, Location, more: https://github.com/MoH-Malaysia/covid19-public/tree/main/epidemic/{specific_file}
# Vaccination: https://github.com/CITF-Malaysia/citf-public/tree/main/vaccination/{specific_file}

url_cases = https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv
url_vax = https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv
url_vax_age = https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_demog_age.csv
